In [71]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import vectormath as vmath

In [72]:
df_original = pd.read_csv('../my_data_processing/20220302Full-tilt/data/20220302Full-tilt-quaternion.csv').iloc[15000:34400,:]
df_original = df_original.reset_index()
df_original = df_original.drop(columns=['Frame', 'Sub Frame', 'Diabolo-RX', 'Diabolo-RY', 'Diabolo-RZ','Diabolo-RW','stick_left-RX','stick_left-RY', 'stick_left-RZ', 'stick_left-RW','stick_right-RX', 'stick_right-RY','stick_right-RZ', 'stick_right-RW'])
# df_original: values from original data recording
df_pitches = pd.read_csv('output_pitch.csv')
# df_pitches: original calculated pitch values and predicted values (using starting pitch, the constant, and predicted angular speed)
df_predicted_values = pd.read_csv('output.csv')
# df_predicted_values: predicted values x,y,z,omega(w) of diabolo based on analytical model
df_angular_speed = pd.read_csv('calculated_angular_speed.csv')
# df_angular_speed: original calculated angular speed values

In [73]:
remove_lines = []
for i in range(len(df_original)):
    line1 = df_original.iloc[i].to_numpy()
    line2 = df_pitches.iloc[i].to_numpy()
    line3 = df_predicted_values.iloc[i].to_numpy()
    line4 = df_angular_speed.iloc[i].to_numpy()
    if np.isnan(line1).any():
        remove_lines.append(i)
        continue
    if np.isnan(line2).any():
        remove_lines.append(i)
        continue
    if np.isnan(line3).any():
        remove_lines.append(i)
        continue
    if np.isnan(line4).any():
        remove_lines.append(i)
        continue
                    

In [74]:
df_original = df_original.drop(index=remove_lines)
df_pitches = df_pitches.drop(index=remove_lines)
df_predicted_values = df_predicted_values.drop(index=remove_lines)
df_angular_speed = df_angular_speed.drop(index=remove_lines)

In [75]:
df_original = df_original.reset_index()
df_pitches = df_pitches.reset_index()
df_predicted_values = df_predicted_values.reset_index()
df_angular_speed = df_angular_speed.reset_index()

In [76]:
df_original.head()

,level_0,index,Diabolo-TX,Diabolo-TY,Diabolo-TZ,stick_left-TX,stick_left-TY,stick_left-TZ,stick_right-TX,stick_right-TY,stick_right-TZ
0,0,15000,-309.946259,751.387329,495.335480,-413.098358,701.136963,979.300659,-205.159988,724.131287,1470.387329
1,1,15001,-314.118774,749.735596,502.981079,-412.917236,701.273560,979.119202,-204.208817,722.109375,1474.996582
2,2,15002,-312.205048,750.657776,502.632751,-412.461884,701.279175,979.262512,-203.175751,720.458862,1478.932495
3,3,15003,-313.746826,750.540161,506.090576,-411.785156,701.385559,979.362183,-202.593079,718.882385,1482.612183
4,6,15006,-310.097260,754.756409,511.749695,-411.027588,701.304382,980.413330,-199.655014,713.183533,1494.946899


In [77]:
df_pitches.head()

,index,pitch,predicted_pitch
0,0,-0.040299,-0.040299
1,1,-0.040301,-0.040308
2,2,-0.040303,-0.040317
3,3,-0.040305,-0.040326
4,6,-0.040311,-0.040353


In [78]:
df_predicted_values.head()

,index,X,Y,Z,W
0,0,-0.309946,0.751387,0.495335,58.010032
1,1,-0.309946,0.751387,0.495322,58.010032
2,2,-0.309946,0.751387,0.495281,58.010032
3,3,-0.309946,0.751387,0.495213,58.010032
4,6,-0.309946,0.751387,0.494845,58.010032


In [79]:
df_angular_speed.head()

,index,W
0,0,71.630356
1,1,71.630356
2,2,71.630356
3,3,71.630356
4,6,58.010032


In [80]:
train_values = []

for i in range(len(df_original)-1):
    line_original = df_original.iloc[i]
    
    # positions
    x_diabolo = vmath.Vector3(
        line_original['Diabolo-TX'],
        line_original['Diabolo-TY'],
        line_original['Diabolo-TZ'])/1000
    x_left = vmath.Vector3(
        line_original['stick_left-TX'],
        line_original['stick_left-TY'],
        line_original['stick_left-TZ'])/1000
    x_right = vmath.Vector3(
        line_original['stick_right-TX'],
        line_original['stick_right-TY'],
        line_original['stick_right-TZ'])/1000
    x_center = (x_left-x_right)/2
    x_diabolo_p = x_diabolo - x_center
    x_left_p = x_left - x_center
    x_right_p = x_right - x_center

    # velocity
    line_original_next = df_original.iloc[i+1]
    x_diabolo_next = vmath.Vector3(
        line_original_next['Diabolo-TX'],
        line_original_next['Diabolo-TY'],
        line_original_next['Diabolo-TZ'])/1000

    v_diabolo = (x_diabolo_next - x_diabolo) * 600

    # next stick values
    x_left_next = vmath.Vector3(
        line_original_next['stick_left-TX'],
        line_original_next['stick_left-TY'],
        line_original_next['stick_left-TZ'])/1000
    x_right_next = vmath.Vector3(
        line_original_next['stick_right-TX'],
        line_original_next['stick_right-TY'],
        line_original_next['stick_right-TZ'])/1000
    x_center_next = (x_left_next-x_right_next)/2
    x_left_next_p = x_left_next - x_center_next
    x_right_next_p = x_right_next - x_center_next

    train_values.append([
        x_diabolo_p.x,
        x_diabolo_p.y,
        x_diabolo_p.z,
        v_diabolo.x,
        v_diabolo.y,
        v_diabolo.z,
        df_angular_speed.iloc[i]['W'],
        x_right_next_p.x,
        x_right_next_p.y,
        x_right_next_p.z,
        x_left_next_p.x,
        x_left_next_p.y,
        x_left_next_p.z,
        x_right_p.x,
        x_right_p.y,
        x_right_p.z,
        x_left_p.x,
        x_left_p.y,
        x_left_p.z
    ])

train_values = np.array(train_values)

In [81]:
df_train = pd.DataFrame(train_values, columns=[
    'x_diabolo_p.x',
    'x_diabolo_p.y',
    'x_diabolo_p.z',
    'v_diabolo.x',
    'v_diabolo.y',
    'v_diabolo.z',
    'w',
    'x_right_next_p.x',
    'x_right_next_p.y',
    'x_right_next_p.z',
    'x_left_next_p.x',
    'x_left_next_p.y',
    'x_left_next_p.z',
    'x_right_p.x',
    'x_right_p.y',
    'x_right_p.z',
    'x_left_p.x',
    'x_left_p.y',
    'x_left_p.z'
])

In [82]:
df_train

,x_diabolo_p.x,x_diabolo_p.y,x_diabolo_p.z,v_diabolo.x,v_diabolo.y,v_diabolo.z,w,x_right_next_p.x,x_right_next_p.y,x_right_next_p.z,x_left_next_p.x,x_left_next_p.y,x_left_next_p.z,x_right_p.x,x_right_p.y,x_right_p.z,x_left_p.x,x_left_p.y,x_left_p.z
0,-0.205977,0.762884,0.740879,-2.503509,-0.991040,4.587359,71.630356,-0.099855,0.732527,1.722935,-0.308563,0.711691,1.227058,-0.101191,0.735628,1.715931,-0.309129,0.712634,1.224844
1,-0.209765,0.760154,0.750920,1.148236,0.553308,-0.208997,71.630356,-0.098533,0.730049,1.728767,-0.307819,0.710869,1.229098,-0.099855,0.732527,1.722935,-0.308563,0.711691,1.227058
2,-0.207562,0.760248,0.752468,-0.925067,-0.070569,2.074695,71.630356,-0.097997,0.727631,1.734237,-0.307189,0.710134,1.230987,-0.098533,0.730049,1.728767,-0.307819,0.710869,1.229098
3,-0.209151,0.759289,0.757716,2.189740,2.529749,3.395471,71.630356,-0.093969,0.719123,1.752214,-0.305341,0.707244,1.237680,-0.097997,0.727631,1.734237,-0.307189,0.710134,1.230987
4,-0.204411,0.760696,0.769016,-0.137365,0.182702,1.305322,58.010032,-0.092688,0.716447,1.757704,-0.304607,0.706382,1.239727,-0.093969,0.719123,1.752214,-0.305341,0.707244,1.237680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18533,-0.426195,0.345136,0.594622,-1.812671,0.708215,-1.958240,41.058010,-0.452624,0.534787,1.427641,-0.410368,0.216190,1.231729,-0.453189,0.535238,1.428019,-0.409650,0.215799,1.231530
18534,-0.428575,0.345895,0.591070,-0.456317,0.101339,0.287494,41.058010,-0.452732,0.534917,1.427829,-0.410453,0.216180,1.231684,-0.452624,0.534787,1.427641,-0.410368,0.216190,1.231729
18535,-0.429346,0.346134,0.591665,2.079584,-0.980832,0.541607,41.058010,-0.452763,0.534994,1.427843,-0.410544,0.216285,1.231856,-0.452732,0.534917,1.427829,-0.410453,0.216180,1.231684
18536,-0.425851,0.344486,0.592489,-0.630597,0.176981,-0.602490,41.058010,-0.452983,0.535053,1.427766,-0.410622,0.216238,1.231756,-0.452763,0.534994,1.427843,-0.410544,0.216285,1.231856


In [243]:
label_values = []
for i in range(len(df_original)-1):
    diff_pitch = df_pitches.iloc[i+1]['pitch'] - df_pitches.iloc[i+1]['predicted_pitch']
    diff_x = df_original.iloc[i+1]['Diabolo-TX']/1000 - df_predicted_values.iloc[i+1]['X']
    diff_y = df_original.iloc[i+1]['Diabolo-TY']/1000 - df_predicted_values.iloc[i+1]['Y']
    diff_z = df_original.iloc[i+1]['Diabolo-TZ']/1000 - df_predicted_values.iloc[i+1]['Z']
    label_values.append([
        diff_x,
        diff_y,
        diff_z,
        diff_pitch
    ])
label_values = np.array(label_values)

In [238]:
#scaler = StandardScaler()
#label_values = scaler.fit_transform(label_values)

In [244]:
train_x, test_x, train_y, test_y = train_test_split(df_train.to_numpy(), label_values, test_size=0.2, random_state=42)

In [245]:
model = keras.Sequential([
    keras.layers.Dense(units=19),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=4, activation='sigmoid')
])
model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError(), metrics=['mse'])

In [246]:
history = model.fit(train_x, train_y, epochs=300, batch_size=100, verbose=0)

In [247]:
model.evaluate(test_x, test_y)

116/116 [==============================] - 0s 397us/step - loss: 0.1321 - mse: 0.1321


[0.13214580714702606, 0.13214580714702606]

In [252]:
model.save('my_model_ll')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model_ll\assets
